### Clean Merge Result

In [ ]:
# !pip install modin[ray]

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 
# Not: Eğer multiprocessing kullanılacaksa ray init kullanılmamalı

In [11]:
#import modin.pandas as pd
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 50)

#### Clean One Character And Digit

In [26]:
df = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result.csv")
df

,sentence,frequency,ratio,cumul_ratio
0,evet,1948596,1.348816,1.348816
1,hayır,1250401,0.865526,2.214343
2,tamam,882921,0.611157,2.825500
3,ne,753105,0.521299,3.346799
4,teşekkürler,343527,0.237789,3.584588
...,...,...,...,...
20975605,omalley bize buzsuz scotch ver,2,0.000001,99.999994
20975606,omalley bu acele ne,2,0.000001,99.999996
20975607,omalley bu milton wing,2,0.000001,99.999997
20975608,omalley buna cevap verecek zamanım,2,0.000001,99.999999


In [ ]:
df = df[~df.iloc[:,0].str.contains(fr"fff", na=True)]
df.reset_index(drop=True, inplace=True)
df

In [27]:
def clean_num(var):
    var_list = re.findall("\w+", var)
    try:
        if var_list[0][0].isdigit() or (len(var_list) <= 1 and len(var_list[0]) <= 1):
            return "digit or one character"
        else:
            return var
    except:
        return "not digit or character"

In [ ]:
df.iloc[:,0] = df.iloc[:,0].apply(lambda x : clean_num(x))
df

In [ ]:
df[df.iloc[:,0] == "digit or one character"]

In [ ]:
df[df.iloc[:,0] == "not digit or character"]

In [ ]:
df.drop(df[df.iloc[:,0] == "digit or one character"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# df.drop(df[df.iloc[:,0] == "not digit or character"].index, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df

In [ ]:
df.to_csv("Clean_Spark_Sentence_Merge_Result_Clean_Digit_One_Char.csv", index=False)

#### Include Name Entity

In [12]:
df_no_digit = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result_Clean_Digit_One_Char.csv")
df_no_digit = df_no_digit[["sentence"]].head(1000000)
df_no_digit

,sentence
0,evet
1,hayır
2,tamam
3,ne
4,teşekkürler
...,...
999995,hemen ateş etme
999996,buraya geldiğim için üzgünüm
999997,üzgünüm buna bakmam lazım
999998,siz onun arkadaşı mısınız


In [13]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [14]:
help(multiprocessing)

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.9/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag

In [15]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [16]:
df_ne = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Name Entity/Special_Name_Unique.xlsx")
df_ne = df_ne.head(5)
df_ne

,spec_name
0,aaron
1,abbas
2,abbey
3,abby
4,abe


In [17]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [18]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
special_name_list = manager.list()

def find_special_name(name_ent):
    name_in_df = df_no_digit[df_no_digit.iloc[:,0].str.contains(fr"(?:\s|^){name_ent}(?:\s|$)", na=True)]
    ne_list = name_in_df.iloc[:,0].to_list()
    for j in ne_list:
        special_name_list.append(j) 

if __name__ == '__main__':
    with Pool(8) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(find_special_name, ne_list) # string_word liste

In [19]:
list_special_name = list(special_name_list)
df_special_name = pd.DataFrame(list_special_name)
df_special_name.rename(columns={0:"name_ent_include"}, inplace=True)
df_special_name

,name_ent_include
0,abbas
1,abbey
2,abbey mount
3,downton abbey
4,carfax abbey
...,...
170,abby dur
171,abby tatlım
172,abby uyanık mısın
173,abby bana bak


In [20]:
df_special_name.to_csv("Name_Entity_Sentence.csv", index=False)

#### Extract Name Entity From Data

In [ ]:
df_no_digit = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result_Clean_Digit_One_Char.csv")
df_no_digit = df_no_digit[["sentence"]].head(1000000)
df_no_digit

#### Name Entity Change Same Name

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

In [ ]:
import modin.pandas as pd

In [21]:
df_special_name = pd.read_csv("Name_Entity_Sentence.csv")
df_special_name

,name_ent_include
0,abbas
1,abbey
2,abbey mount
3,downton abbey
4,carfax abbey
...,...
170,abby dur
171,abby tatlım
172,abby uyanık mısın
173,abby bana bak


In [22]:
df_ne = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Name Entity/Special_Name_Unique.xlsx")
df_ne = df_ne.head(5)
df_ne

,spec_name
0,aaron
1,abbas
2,abbey
3,abby
4,abe


In [ ]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [23]:
def replace_name(x):
    for i in ne_list:
        #x = re.sub(fr"(?:\s|^){i}", " ali ", x)
        x = re.sub(fr"(?:\s|^){i}(?:\s|$)", " fikret ", x)
        x = x.strip()        
    return x

In [24]:
df_special_name.iloc[:,0] = df_special_name.iloc[:,0].apply(lambda x : replace_name(x))
df_special_name

,name_ent_include
0,fikret
1,fikret
2,fikret mount
3,downton fikret
4,carfax fikret
...,...
170,fikret dur
171,fikret tatlım
172,fikret uyanık mısın
173,fikret bana bak


In [ ]:
df_special_name.to_csv("Name_Entity_Change_Sentence.csv", index=False)